# Amplitude model with LS-couplings

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import logging
import os

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import sympy as sp
from ampform_dpd import (
    AmplitudeModel,
    _get_coupling_base,  # pyright:ignore[reportPrivateUsage]
)
from ampform_dpd.decay import Particle
from ampform_dpd.io import aslatex, cached, simplify_latex_rendering
from IPython.display import Latex
from matplotlib_inline.backend_inline import set_matplotlib_formats
from sympy.core.symbol import Str
from tensorwaves.interface import Function
from tqdm.auto import tqdm

from polarimetry.data import (
    create_data_transformer,
    generate_meshgrid_sample,
    generate_phasespace_sample,
)
from polarimetry.function import integrate_intensity, sub_intensity
from polarimetry.io import display_latex, mute_jax_warnings
from polarimetry.lhcb import (
    get_conversion_factor_ls,
    load_model_builder,
    load_model_parameters,
)
from polarimetry.lhcb.particle import load_particles
from polarimetry.plot import use_mpl_latex_fonts

plt.rc("font", size=13)
mute_jax_warnings()
set_matplotlib_formats("svg")
simplify_latex_rendering()
use_mpl_latex_fonts()

MODEL_FILE = "../../data/model-definitions.yaml"
PARTICLES = load_particles("../../data/particle-definitions.yaml")

NO_LOG = "EXECUTE_NB" in os.environ
if NO_LOG:
    logging.disable(logging.CRITICAL)

## Model inspection

In [ ]:
def formulate_model(title: str) -> AmplitudeModel:
    builder = load_model_builder(MODEL_FILE, PARTICLES, title)
    imported_parameters = load_model_parameters(
        MODEL_FILE, builder.decay, title, PARTICLES
    )
    model = builder.formulate()
    model.parameter_defaults.update(imported_parameters)
    return model


def simplify_notation(expr: sp.Expr) -> sp.Expr:
    def substitute_node(node):
        if isinstance(node, sp.Indexed) and node.indices[2:] == (0, 0):
            return sp.Indexed(node.base, *node.indices[:2])
        return node

    substitutions = {n: substitute_node(n) for n in sp.preorder_traversal(expr)}
    return cached.xreplace(expr, substitutions)


LS_MODEL = formulate_model("Alternative amplitude model obtained using LS couplings")
simplify_notation(LS_MODEL.intensity.args[0].args[0].args[0].cleanup())

In [ ]:
display_latex({simplify_notation(k): v for k, v in LS_MODEL.amplitudes.items()})

In [ ]:
H_prod = _get_coupling_base(helicity_basis=False, typ="production")

latex = R"""
\begin{array}{c|ccc|c}
  \textbf{Decay} & \textbf{coupling} & & & \textbf{factor} \\
  \hline
"""
for chain in LS_MODEL.decay.chains:
    R = Str(chain.resonance.latex)
    L = chain.incoming_ls.L
    S = chain.incoming_ls.S
    symbol = H_prod[R, L, S]
    value = sp.sympify(LS_MODEL.parameter_defaults[symbol])
    factor = get_conversion_factor_ls(chain.resonance, L, S)
    coupling_value = f"{aslatex(symbol)} &=& {aslatex(value.n(3))}"
    latex += Rf"  {aslatex(chain)} & {coupling_value} & {factor:+d} \\" "\n"
latex += R"\end{array}"
Latex(f"{latex}")

It is asserted that these amplitude expressions to not evaluate to $0$ once the Clebsch-Gordan coefficients are evaluated.

In [ ]:
def assert_non_zero_amplitudes(model: AmplitudeModel) -> None:
    for amplitude in tqdm(model.amplitudes.values(), disable=NO_LOG):
        assert cached.doit(amplitude) != 0


assert_non_zero_amplitudes(LS_MODEL)

:::{seealso}
See {ref}`amplitude-model:Resonances and LS-scheme` for the allowed $LS$-values.
:::

## Distribution

In [ ]:
def lambdify(model: AmplitudeModel) -> sp.Expr:
    intensity_expr = cached.unfold(model)
    pars = model.parameter_defaults
    free_parameters = {s: v for s, v in pars.items() if "production" in str(s)}
    fixed_parameters = {s: v for s, v in pars.items() if s not in free_parameters}
    subs_intensity_expr = cached.xreplace(intensity_expr, fixed_parameters)
    return cached.lambdify(subs_intensity_expr, free_parameters)


DEFAULT_MODEL = formulate_model("Default amplitude model")
DEFAULT_INTENSITY_FUNC = lambdify(DEFAULT_MODEL)
LS_INTENSITY_FUNC = lambdify(LS_MODEL)

In [ ]:
GRID = generate_meshgrid_sample(DEFAULT_MODEL.decay, resolution=1_000)
TRANSFORMER = create_data_transformer(DEFAULT_MODEL)
GRID.update(TRANSFORMER(GRID))

In [ ]:
def compare_2d_distributions() -> None:
    DEFAULT_INTENSITIES = compute_normalized_intensity(DEFAULT_INTENSITY_FUNC)
    LS_INTENSITIES = compute_normalized_intensity(LS_INTENSITY_FUNC)
    max_intensity = max(
        jnp.nanmax(DEFAULT_INTENSITIES),
        jnp.nanmax(LS_INTENSITIES),
    )
    fig, axes = plt.subplots(
        dpi=200,
        figsize=(7, 4),
        ncols=2,
        sharey=True,
    )
    fig.subplots_adjust(bottom=0, left=0, right=1, top=1, wspace=0.05)
    fig.patch.set_color("none")
    for ax in axes:
        ax.patch.set_color("none")
        ax.set_aspect("equal")
        ax.set_xlabel(s_labels[2])
    ax1, ax2 = axes
    ax1.set_ylabel(s_labels[1])
    ax1.set_title("default model")
    ax2.set_title("LS-model", size=13)
    mesh = ax1.pcolormesh(
        GRID["sigma2"],
        GRID["sigma1"],
        DEFAULT_INTENSITIES,
        rasterized=True,
        vmin=0,
        vmax=max_intensity,
    )
    mesh = ax2.pcolormesh(
        GRID["sigma2"],
        GRID["sigma1"],
        LS_INTENSITIES,
        rasterized=True,
        vmin=0,
        vmax=max_intensity,
    )
    cbar = fig.colorbar(mesh, ax=axes, pad=+0.01, shrink=0.4)
    cbar.ax.set_ylabel("intensity (a.u.)")
    plt.show()


def compute_normalized_intensity(func: Function) -> jax.Array:
    intensities = func(GRID)
    integral = jnp.nansum(intensities)
    return intensities / integral


s_labels = {
    1: R"$M^2\left(K^-\pi^+\right)$ [GeV$^2$]",
    2: R"$M^2\left(pK^-\right)$ [GeV$^2$]",
    3: R"$M^2\left(p\pi^+\right)$ [GeV$^2$]",
}
compare_2d_distributions()

In [ ]:
def visualize_difference() -> None:
    default_intensities = DEFAULT_INTENSITY_FUNC(GRID)
    ls_intensities = compute_normalized_intensity(LS_INTENSITY_FUNC) * jnp.nansum(
        default_intensities
    )
    rel_default_intensities = default_intensities / jnp.nanmax(default_intensities)
    rel_ls_intensities = ls_intensities / jnp.nanmax(default_intensities)
    difference = rel_default_intensities - rel_ls_intensities
    fig, ax = plt.subplots(dpi=500, figsize=(6, 3))
    fig.patch.set_color("none")
    ax.patch.set_color("none")
    ax.set_aspect("equal")
    ax.set_xlabel(s_labels[2])
    ax.set_ylabel(s_labels[1])
    ax.set_xlim(2.02, 4.65)
    ax.set_ylim(0.38, 1.84)
    zmax = jnp.nanmax(jnp.abs(difference))
    zmax = 0.03
    mesh = ax.pcolormesh(
        GRID["sigma2"],
        GRID["sigma1"],
        difference,
        rasterized=True,
        vmin=-zmax,
        vmax=+zmax,
        cmap=plt.cm.coolwarm,
    )
    cbar = fig.colorbar(mesh, pad=+0.01, shrink=0.94)
    cbar.ax.set_ylabel("rel. intensity difference", labelpad=-15, y=0.53)
    cbar.ax.set_yticks([-zmax, 0, +zmax])
    cbar.ax.set_yticklabels([f"${-zmax:+g}$", "$0$", f"${+zmax:+g}$"])
    fig.savefig("../_images/intensity-difference-ls-model.svg", bbox_inches="tight")
    plt.show()


visualize_difference()

## Decay rates

In [ ]:
def compute_decay_rates() -> dict[Particle, tuple[float, float]]:
    decay_rates = {}
    default_I_tot = integrate_intensity(DEFAULT_INTENSITY_FUNC(PHSP))
    LS_I_tot = integrate_intensity(LS_INTENSITY_FUNC(PHSP))
    for chain in tqdm(DEFAULT_MODEL.decay.chains, disable=NO_LOG):
        filter_ = [chain.resonance.latex]
        LS_I_sub = sub_intensity(LS_INTENSITY_FUNC, PHSP, filter_)
        default_I_sub = sub_intensity(DEFAULT_INTENSITY_FUNC, PHSP, filter_)
        decay_rates[chain.resonance] = (
            float(default_I_sub / default_I_tot),
            float(LS_I_sub / LS_I_tot),
        )
    return decay_rates


PHSP = generate_phasespace_sample(DEFAULT_MODEL.decay, n_events=100_000, seed=0)
PHSP = TRANSFORMER(PHSP)
DECAY_RATES = compute_decay_rates()
src = R"""
\begin{array}{l|rr|r}
  \textbf{Resonance} & \textbf{Default} & \textbf{LS-model} & \textbf{Difference}\\
  \hline
"""
for res, (default_rate, ls_rate) in DECAY_RATES.items():
    default_rate *= 100
    ls_rate *= 100
    src += (
        Rf"  {res.latex} & {default_rate:.2f} & {ls_rate:.2f} &"
        rf" {ls_rate - default_rate:+.2f} \\"
        "\n"
    )
    del res, default_rate, ls_rate
src += R"\end{array}"
Latex(src)

In [ ]:
from deepdiff import DeepDiff

actual = {p.name: val for p, (val, err) in DECAY_RATES.items()}
expected = {
    "L(1405)": 0.07778927301796905,
    "L(1520)": 0.019140409080481712,
    "L(1600)": 0.05156508524716808,
    "L(1670)": 0.01152765176461739,
    "L(1690)": 0.011620347822014116,
    "L(2000)": 0.09545104884536688,
    "D(1232)": 0.28734702629964115,
    "D(1600)": 0.04499512444857963,
    "D(1700)": 0.03886137420714059,
    "K(700)": 0.02989528640855732,
    "K(892)": 0.21953853300085216,
    "K(1430)": 0.14700511274990016,
}
diff = DeepDiff(actual, expected, significant_digits=16)
assert not diff, diff.pretty()

:::{tip}
Compare with the values with uncertainties as reported in {ref}`uncertainties:Decay rates`.
:::